First, make sure the package is installed.

In [3]:
! pip install diavgeia-api


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


And initiate our main object, the client:

In [4]:
from diavgeia_api import DiavgeiaClient

# Initialize client
client = DiavgeiaClient()

# We will recycle it throughout the notebook

And take a look at the available methods:

In [5]:
# Get all methods that don't start with underscore (public methods)
public_methods = [method for method in dir(client) if not method.startswith('_') and callable(getattr(client, method))]

print("Available methods:")
for method in sorted(public_methods):
    # Print the method name and its docstring if available
    doc = getattr(client, method).__doc__
    if doc:
        # Get just the first line of the docstring for a brief description
        brief_doc = doc.strip().split('\n')[0]
        print(f"- {method}: {brief_doc}")
    else:
        print(f"- {method}")

Available methods:
- get_a_decision: Return a specific decision's details.
- get_a_decisions_specific_version: Returns details of a specific version of a decision.
- get_a_decisions_version_log: Returns details of a specific version of a decision.
- get_a_types_details: Returns details of a type.
- get_a_types_summary: Returns a summary of a specific type.
- get_all_types: Returns all existing types.
- get_dictionaries: Return the list of available dictionaries.
- get_dictionary: Return all items for a specific dictionary.
- get_organization: Returns details of a specific organization.
- get_organization_positions: Returns positions of a specific organization.
- get_organization_signers: Returns signers of a specific organization.
- get_organization_units: Returns units of a specific organization.
- get_organizations: Returns a list of registered organizations.
- search_advanced: Search for decisions with advanced query syntax.
- search_decisions: Search for decisions with simple param

Then, let's target one specific decision (99Θ346ΜΓΨ7-Κ52 in our example) and explore.

In [6]:

search_for_decision_id = "99Θ346ΜΓΨ7-Κ52"

decision = client.get_a_decision(search_for_decision_id)

In [7]:
print(f"Protocol Number: {decision.protocolNumber}")
print(f"Subject: {decision.subject}")
print(f"Submission Timestamp: {decision.submissionTimestamp}")
print(f"Total Amount to pay: {decision.extraFieldValues.amountWithVAT}")
print(f"Amount from which buget: {decision.extraFieldValues.budgettype}")

Protocol Number: 2829
Subject: Έγκριση δέσμευσης πίστωσης ΑΛΕ 2420301001 για την πληρωμή ισόποσης δαπάνης σε βάρος της πίστωσης του προϋπολογισμού εξόδων του Ειδικού Φορέα 1004 201-0000000.
Submission Timestamp: 2025-04-17 08:21:26.446000+00:00
Total Amount to pay: amount=2480.0 currency='EUR'
Amount from which buget: Τακτικός Προϋπολογισμός


Which organization produced this decision?

In [8]:
org_of_this_decision = decision.organizationId
print(f"Organization issuing the decision: {org_of_this_decision}")

Organization issuing the decision: 100048574


Since a unique identifier tells nothing, we're going to use the `get_organization` method to retrieve information about it:

In [9]:
organization_that_produced_it = client.get_organization(org_of_this_decision)

In [10]:
organization_that_produced_it

Organization(uid='100048574', latinName='PRESIDENCY_OF_THE_GOVERNMENT', abbreviation=None, label='ΠΡΟΕΔΡΙΑ ΤΗΣ ΚΥΒΕΡΝΗΣΗΣ', status=<OrganizationStatus.ACTIVE: 'active'>, category='MINISTRY', vatNumber='996887930', fekNumber='133', fekIssue='fektype_A', fekYear='2019', organizationDomains=['GovernanceInstrumentation'], website=None, supervisorOrgUid=None, supervisorOrgName=None)

In [11]:
print(f"Organization with id: {organization_that_produced_it.uid} is ** {organization_that_produced_it.label} ** ")
print(f"Has a vat number of {organization_that_produced_it.vatNumber} ")
print(f"And is part of the category: {organization_that_produced_it.category} ")


Organization with id: 100048574 is ** ΠΡΟΕΔΡΙΑ ΤΗΣ ΚΥΒΕΡΝΗΣΗΣ ** 
Has a vat number of 996887930 
And is part of the category: MINISTRY 


But which specific unit of it produced it?
Let's use the `get_organization_units` to learn more about this organization and its units.

In [12]:
unit_of_this_decision = decision.unitIds
print(f"Unit issuing the decision: {unit_of_this_decision}")

Unit issuing the decision: ['100062183']


In [13]:
units_of_organization_that_produced_it = client.get_organization_units(organization_id = org_of_this_decision)

In [14]:
the_unit_we_are_looking_for = [ 
                            unit 
                            for unit in 
                            units_of_organization_that_produced_it.units 
                            if unit.uid in unit_of_this_decision 
                            ]

In [15]:
print(the_unit_we_are_looking_for)

[Unit(uid='100062183', label='ΜΟΝΑΔΑ ΔΙΟΙΚΗΤΙΚΗΣ ΚΑΙ ΟΙΚΟΝΟΜΙΚΗΣ ΥΠΟΣΤΗΡΙΞΗΣ', active=True, activeFrom=datetime.datetime(2019, 9, 7, 0, 0, tzinfo=TzInfo(UTC)), activeUntil=None, category='GENERAL_ADMINISTRATION', unitDomains=None, parentId='100048985')]


There's also information about who signed this expense. 

In [16]:
signer_of_this_decision = decision.signerIds
print(f"Signer: {signer_of_this_decision}")

Signer: ['100064303']


Let's use the `get_organization_signers`  to find who exactly approved this expense.

In [17]:
signers_of_organization_that_produced_it = client.get_organization_signers(organization_id = org_of_this_decision)

In [18]:
the_signer_we_are_looking_for = [ 
                            singer 
                            for singer in 
                            signers_of_organization_that_produced_it.signers 
                            if singer.uid in signer_of_this_decision 
                            ][0]

In [19]:
signer_message = f"""Signer with id: {the_signer_we_are_looking_for.uid} is Ms {the_signer_we_are_looking_for.firstName} {the_signer_we_are_looking_for.lastName} and holds the position of {the_signer_we_are_looking_for.units[0].positionLabel}"""
print(signer_message)

Signer with id: 100064303 is Ms ΚΩΝΣΤΑΝΤΙΝΑ ΚΑΖΑΚΟΥ and holds the position of Διευθυντής
